In [ ]:
import itertools
from collections import defaultdict

import pandas as pd

In [ ]:
classifiers = ['decision_tree', 'random_forest', 'knn', 'naive_bayes', 'mlp', 'svm']
sections = ['full', 'time_decomposition']
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
genre_pairs = list(itertools.combinations(genres, 2))

use_weighted_voting = False

In [ ]:
for classifier in classifiers:
    for split in range(5):
        for section in sections:
            genre_pair_results = {}
            for genre_pair in genre_pairs:
                genre_pair_results[genre_pair] = pd.read_csv(f'./predictions_rr/all/{classifier}_rr_{split}_{section}_{genre_pair[0]}_{genre_pair[1]}.csv', index_col=['file','section'])

            combined_prediction = pd.DataFrame(genre_pair_results[tuple(genre_pair)]['genre'], index=genre_pair_results[tuple(genre_pair)].index, columns=['genre', 'predicted', 'score'])
            for i in genre_pair_results[tuple(genre_pair)].index:
                weighted_votes = defaultdict(float)
                for genre_pair, results in genre_pair_results.items():
                    if classifier == 'svm':
                        weighted_votes[results.loc[i, 'predicted']] += abs(results.loc[i, 'score']) if use_weighted_voting else 1
                    else:
                        weighted_votes[results.loc[i, 'predicted']] += results.loc[i, 'probability'] if use_weighted_voting else 1
                weighted_votes = sorted(list(weighted_votes.items()), key=lambda x: x[1], reverse=True)
                combined_prediction.loc[i, 'predicted'] = weighted_votes[0][0]
                combined_prediction.loc[i, 'score'] = weighted_votes[0][1] / sum(x[1] for x in weighted_votes)
                
            combined_prediction.to_csv(f'./predictions_rr/{classifier}_rr_{split}_{section}.csv')